In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import time

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36")

In [3]:
browser = webdriver.Chrome(options = chrome_options)

browser.get('https://www.youtube.com/watch?v=cGPpUdaISGE')

In [4]:
scroll_pause_time = 3
last_height = browser.execute_script("return document.documentElement.scrollHeight")

while True:
    browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = browser.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [5]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [6]:
data = []

for s in soup.find_all('ytd-comment-thread-renderer'):
    try:
        account = s.find('span', class_ = 'style-scope ytd-comment-view-model style-scope ytd-comment-view-model').text.strip()
        comment = s.find('span', class_ = 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap').text.replace('\n','')
        comment = comment.replace(".", ". ")
        data.append([account, comment])
    except:
        continue

In [7]:
browser.quit()

In [8]:
data

[['@smarthbakshi7488', 'A long way to go! '],
 ['@ChocolateCloud3112',
  'I am little below 20 lpa now.  Hope to cross this soon'],
 ['@pahadi_soul6657', 'Front-End developer roadmap ️'],
 ['@kunjshelat2318', 'Sir how did you learn java development?'],
 ['@abhinandankumar6437', 'What about fresher is any opportunity for fresher'],
 ['@studies443', 'Hindi language available in classes'],
 ['@ahsan769', 'Eligibility criteria kya tha company me jaane k liye'],
 ['@divyanshsinghal3917',
  'Sir ji me 2 saal se Wipro me kaam kar rha hu. Abhi tak ek bhi enhancement nhi kiya and abhi job change karne me problem aa rhi he.  Apni roles and responsibilities kese explain Karu. Please help me'],
 ['@The_Shubham_Soni',
  'Paahji mere pas 2 offer h ek TCS Or ek Dusri company me.  Konsi thik rahegi?']]

In [9]:
import pandas as pd

df = pd.DataFrame(data, columns = ['user', 'comment'])

In [10]:
df

,user,comment
0,@smarthbakshi7488,A long way to go!
1,@ChocolateCloud3112,I am little below 20 lpa now. Hope to cross t...
2,@pahadi_soul6657,Front-End developer roadmap ️
3,@kunjshelat2318,Sir how did you learn java development?
4,@abhinandankumar6437,What about fresher is any opportunity for fresher
5,@studies443,Hindi language available in classes
6,@ahsan769,Eligibility criteria kya tha company me jaane ...
7,@divyanshsinghal3917,Sir ji me 2 saal se Wipro me kaam kar rha hu. ...
8,@The_Shubham_Soni,Paahji mere pas 2 offer h ek TCS Or ek Dusri c...


## Preprocessing

In [11]:
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='auto', target='en')

df['comment'] = df['comment'].apply(lambda x: translator.translate(x))

In [12]:
df

,user,comment
0,@smarthbakshi7488,A long way to go!
1,@ChocolateCloud3112,I am little below 20 lpa now. Hope to cross t...
2,@pahadi_soul6657,Front-End developer roadmap ️
3,@kunjshelat2318,Sir how did you learn java development?
4,@abhinandankumar6437,What about fresher is any opportunity for fresher
5,@studies443,Hindi language available in classes
6,@ahsan769,What was the eligibility criteria to join the ...
7,@divyanshsinghal3917,Sir ji I have been working in Wipro for 2 year...
8,@The_Shubham_Soni,"Paahji, I have 2 offers, one from TCS and one ..."


In [28]:
from nltk.corpus import stopwords
import string
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [29]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))
    
    return " ".join(text)

In [30]:
print(transform_text('HEY, MY name IS prajwal.'))

hey name prajwal


In [31]:
df['comment'] = df['comment'].apply(transform_text)

In [33]:
df.head()

,user,comment
0,@smarthbakshi7488,long way go
1,@ChocolateCloud3112,little 20 lpa hope cross soon
2,@pahadi_soul6657,developer roadmap
3,@kunjshelat2318,sir learn java development
4,@abhinandankumar6437,fresher opportunity fresher
